In [1]:
from collections import defaultdict
import json
import os

from dask.distributed import Client
import fsspec
import numpy as np
import pandas as pd
from shapely.ops import cascaded_union
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import RadiusNeighborsClassifier

## And a bunch of carbonplan dependencies
from carbonplan_data import cat as core_cat

from carbonplan_retro.data import cat
from carbonplan_retro.analysis.assign_project_fldtypcd import load_classification_data
from carbonplan_retro.load.geometry import (
    get_overlapping_states,
    load_supersections,
)
from carbonplan_retro.load.project_db import load_project_db

In [2]:
def get_aoi(ss_ids):
    da = core_cat.nlcd.raster(region="conus").to_dask()
    crs = da.attrs["crs"]

    supersections = load_supersections().to_crs(crs)

    subset_supersection = supersections[supersections["ss_id"].isin(ss_ids)].copy()
    subset_supersection.loc[:, "dissolve_all"] = 1

    aoi = subset_supersection.dissolve(by="dissolve_all").buffer(150_000).to_crs("epsg:4326").item()
    return aoi

In [3]:
def species_array_to_d(species_array):
    return {str(species["code"]): round(species["fraction"], 4) for species in species_array}


def prepare_regional_classifier(ss_ids, prefit_radius=None):
    """returns trained classifier and data vectorizer to apply to multiple opr_ids"""
    
    if (len(ss_ids) == 1) & (ss_ids[0] > 200):
        data = load_classification_data(["ak"])

    else:
        da = core_cat.nlcd.raster(region="conus").to_dask()
        crs = da.attrs["crs"]

        supersections = load_supersections().to_crs(crs)

        subset_supersection = supersections[supersections["ss_id"].isin(ss_ids)].copy()
        subset_supersection.loc[:, "dissolve_all"] = 1

        aoi = (
            subset_supersection.dissolve(by="dissolve_all")
            .buffer(150_000)
            .to_crs("epsg:4326")
            .item()
        )

        postal_codes = get_overlapping_states(aoi)
        print(f"preparing to load: {[x for x in postal_codes]}")
        data = load_classification_data(postal_codes, aoi=aoi)

    print(f"fitting classifier ")
    if prefit_radius:
        print(f"using cached radius: {prefit_radius}")
        clf = RadiusNeighborsClassifier(
            weights="distance", algorithm="brute", outlier_label=-999, radius=prefit_radius
        )
    else:
        # JJH: check algorthim
        base_clf = RadiusNeighborsClassifier(
            weights="distance", algorithm="brute", outlier_label=-999
        )
        param_grid = [
            {"radius": np.arange(0.15, 0.651, 0.025)}
        ]  # initial testing never yielded a case where we went above 0.5

        # we can parallalize this further
        # dask_ml.model_selection.GridSearchCV
        clf = GridSearchCV(
            base_clf, param_grid, n_jobs=int(os.cpu_count() / 2), cv=5, refit=True, verbose=10
        )

    clf.fit(data["features"], data["targets"])
    return clf, data["dictvectorizer"]

In [4]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:36525 Dashboard: /user/jhamman/proxy/8787/status,Cluster Workers: 4 Cores: 8 Memory: 53.69 GB


In [5]:
project_db = load_project_db('Forest-Offset-Projects-v0.3', save=False)
projects = project_db[~project_db["project"]["early_action"].str.startswith("CAR")]
projects = projects[~project_db["project"]["species"].isnull()]
projects = projects[~projects["project"]["species"].apply(lambda x: "all" in x)]

loading load Forest-Offset-Projects-v0.3 from /home/jovyan/retro/data
failed to load from disk -- grabbing Forest-Offset-Projects-v0.3 from google


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
retro_json = cat.retro_db_light_json.read()

In [7]:
clf_cache = {}  # separate cell so you can re-run next cell if anything bonks

In [11]:
project["project"]["supersection_ids"].item()

[55, 79]

In [17]:
# classifications = defaultdict(dict)

USE_CACHE = False

if USE_CACHE:
    with fsspec.open(
        "az://carbonplan-scratch/radius_neighbor_params.json",
        account_name="carbonplan",
        mode="r",
        account_key=os.environ["BLOB_ACCOUNT_KEY"],
    ) as f:
        radius_params = json.load(f)
else:
    radius_params = {}
supersections = set(tuple(project["project"]["supersection_ids"].item()) for _, project in projects.iterrows())
for ssid in supersections:
    print(ssid)
#     if opr_id in ["CAR1094", "CAR1032"]:
#         print(f"skipping {opr_id} -- discuss w group")
#         continue

#     print(opr_id)
#     cache_key = project["project"]["supersection_ids"].astype(str).item()
#     try:
#         clf, data_encoder = clf_cache[cache_key]
#     except:
    clf, data_encoder = prepare_regional_classifier(ssid)  #, radius_params.get(cache_key)
#     )
#     clf_cache[project["project"]["supersection_ids"].astype(str).item()] = (clf, data_encoder)

#     for aa_id, species_arr in project_db["project"]["species"][opr_id].items():

#         feat_dict = species_array_to_d(species_arr)
#         feats = data_encoder.transform(feat_dict)
#         classification = pd.Series(clf.predict_proba(feats).flatten(), index=clf.classes_)
#         classifications[opr_id][aa_id] = classification[classification > 0].sort_values().to_dict()

(32,)
preparing to load: ['ar', 'la', 'tx', 'al', 'fl', 'ga', 'ms']


ValueError: cannot reindex from a duplicate axis

distributed.client - ERROR - Failed to reconnect to scheduler after 600.00 seconds, closing client
distributed.utils - ERROR - Timed out trying to connect to 'tcp://127.0.0.1:36525' after 600 s: Timed out trying to connect to 'tcp://127.0.0.1:36525' after 600 s: in <distributed.comm.tcp.TCPConnector object at 0x7f017fe6e390>: ConnectionRefusedError: [Errno 111] Connection refused
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/comm/tcp.py", line 186, in read
    n_frames = await stream.read_bytes(8)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/client.py", line 1223, in _handle_report
    msgs = await self.scheduler_comm.comm.read()
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/comm/tcp.py", line 201, in read
    conv

In [ ]:
%debug

> /srv/conda/envs/notebook/lib/python3.7/site-packages/pandas/core/indexes/base.py(3476)_can_reindex()
   3474         # trying to reindex on an axis with duplicates
   3475         if not self._index_as_unique and len(indexer):
-> 3476             raise ValueError("cannot reindex from a duplicate axis")
   3477 
   3478     def reindex(self, target, method=None, level=None, limit=None, tolerance=None):



ipdb>  u


> /srv/conda/envs/notebook/lib/python3.7/site-packages/pandas/core/internals/managers.py(1301)reindex_indexer()
   1299         # some axes don't allow reindexing with dups
   1300         if not allow_dups:
-> 1301             self.axes[axis]._can_reindex(indexer)
   1302 
   1303         if axis >= self.ndim:



ipdb>  u


> /srv/conda/envs/notebook/lib/python3.7/site-packages/pandas/core/generic.py(4880)_reindex_with_indexers()
   4878                 fill_value=fill_value,
   4879                 allow_dups=allow_dups,
-> 4880                 copy=copy,
   4881             )
   4882             # If we've made a copy once, no need to make another one



...

                     CN           PLT_CN  CONDID  OWNCD  FORTYPCD  FLDTYPCD  \
index                                                                         
107237  259578802010854  249672510010854       2   46.0     503.0     503.0   
107238  259577851010854  249672511010854       1   46.0     406.0     161.0   
107239  259577852010854  249672511010854       2   46.0     161.0     161.0   
107240  259574166010854  249672512010854       1   46.0     406.0     706.0   
107241  259574167010854  249672512010854       2   46.0     406.0     161.0   
...                 ...              ...     ...    ...       ...       ...   
95942    53219663020004  259119059010854       1   11.0     141.0     141.0   
95943    53219721020004  259119060010854       1   11.0     161.0     406.0   
95944    53219814020004  259119061010854       1   11.0     161.0     161.0   
95945    53219915020004  259119062010854       1   11.0     161.0     161.0   
95946    53220006020004  259119063010854       1   1

ipdb>  tree_features


PLT_CN           CONDID
17479078010478   1         {'491': 0.0214, '611': 0.2605, '721': 0.261, '...
17479844010478   1         {'461': 0.1769, '544': 0.2474, '641': 0.0756, ...
17479968010478   2             {'131': 0.8397, '611': 0.1298, '831': 0.0305}
17480533010478   1         {'391': 0.0932, '611': 0.3546, '68': 0.115, '8...
17481234010478   1             {'110': 0.2316, '131': 0.7588, '824': 0.0096}
                                                 ...                        
549073062126144  1         {'316': 0.0634, '401': 0.0195, '407': 0.1066, ...
549073097126144  1         {'221': 0.4001, '544': 0.0554, '551': 0.0276, ...
549073150126144  1         {'316': 0.034, '408': 0.1531, '409': 0.0916, '...
549073158126144  1         {'131': 0.1723, '521': 0.0139, '552': 0.0268, ...
558391170126144  1         {'131': 0.0831, '316': 0.1004, '403': 0.1726, ...
Name: fraction_species, Length: 202146, dtype: object


ipdb>  tree_features.index


MultiIndex([( 17479078010478, 1),
            ( 17479844010478, 1),
            ( 17479968010478, 2),
            ( 17480533010478, 1),
            ( 17481234010478, 1),
            ( 17481349010478, 1),
            ( 17481713010478, 2),
            ( 17481793010478, 1),
            ( 17481964010478, 1),
            ( 17482868010478, 1),
            ...
            (549073022126144, 1),
            (549073023126144, 1),
            (549073036126144, 1),
            (549073037126144, 1),
            (549073040126144, 1),
            (549073062126144, 1),
            (549073097126144, 1),
            (549073150126144, 1),
            (549073158126144, 1),
            (558391170126144, 1)],
           names=['PLT_CN', 'CONDID'], length=202146)


## Store some outputs

Store the 5-fold CV radius parameter as well as the classifications. Can't imagine we'd ever need to
use the radii, but they're sort of expensive to compute so store for good measure.


In [ ]:
fit_radii = {k: v[0].best_params_["radius"] for k, v in clf_cache.items()}

if not USE_CACHE:
    with fsspec.open(
        "az://carbonplan-scratch/radius_neighbor_params.json",
        account_name="carbonplan",
        mode="w",
        account_key=os.environ["BLOB_ACCOUNT_KEY"],
    ) as f:
        json.dump(fit_radii, f, indent=2)

with fsspec.open(
    "az://carbonplan-scratch/project_radius_classification.json",
    account_name="carbonplan",
    mode="w",
    account_key=os.environ["BLOB_ACCOUNT_KEY"],
) as f:
    json.dump(classifications, f, indent=2)